# Use ZTF DR with TAPE

In [1]:
# First, install package to load the data as dask dataframes
# python -mpip install git+https://github.com/hombit/load_ztfdr_for_tape

In [2]:
from itertools import chain
from pathlib import Path

import light_curve as licu
from dask.distributed import Client
from load_ztfdr_for_tape import load_object_source_frames_from_path
from tape import Ensemble, ColumnMapper

### Get Dask dataframes

They are not going to be read right now, we are just planning

In [3]:
%%time

# This is PSC path to the whole DR.
ztf_dr_path = Path('/ocean/projects/phy210048p/shared/hipscat/raw/ztf-dr19/')

# Here we would use a single field only
# For dense fields, like 807, a Galactic one, we are out of memory
# So let's use extragalactic field 795 here
data_path = ztf_dr_path / '0' / 'field000795'

objects, sources = load_object_source_frames_from_path(data_path)

CPU times: user 1.48 s, sys: 209 ms, total: 1.69 s
Wall time: 1.27 s


### Create Dask client
It would allow us to run things in parallel

In [4]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 16
Total threads: 128,Total memory: 247.07 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39425,Workers: 16
Dashboard: http://127.0.0.1:8787/status,Total threads: 128
Started: Just now,Total memory: 247.07 GiB
Comm: tcp://127.0.0.1:35581,Total threads: 8
Dashboard: http://127.0.0.1:37833/status,Memory: 15.44 GiB
Nanny: tcp://127.0.0.1:38911,


### Create TAPE Ensemble and plan analysis

Again, we do it lazily here

In [5]:
ens = Ensemble(client=client)
column_mapper = ColumnMapper(
    id_col='objectid',
    time_col='hmjd',
    flux_col='mag',
    err_col='magerr',
    band_col='filterid',
)
ens.from_dask_dataframe(
    object_frame=objects,
    source_frame=sources,
    column_mapper=column_mapper,
    # Do not make an initial sync of the tables
    sync_tables=False,
    # We did sort the tables by objectid with load_ztfdr_for_tape
    sorted=True,
    sort=False,
)

/jet/home/malanche/.virtualenvs/tape/lib/python3.9/site-packages/dask/dataframe/core.py:5400: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(
/jet/home/malanche/.virtualenvs/tape/lib/python3.9/site-packages/dask/dataframe/core.py:5400: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(


In [6]:
# Filter "bad" detections
ens.source.query("catflags == 0 & magerr > 0.0").update_ensemble()

# Count detections per object and filter by it
ens.calc_nobs()
ens.object.query("nobs_total >= 100")

features = ens.batch(licu.Amplitude(), band_to_calc=None, compute=False,
                     label='features')
max_amplitude = features['amplitude'].max()

Temporary columns dropped from Object Table: ['nobs_total']


### Run the pipeline

`.compute()` would run the computational graph and output the result

In [ ]:
%%time
result = max_amplitude.compute()

result

### Turn Dask cluster off

In [ ]:
client.close()